In [4]:
import serial
import pandas as pd
from datetime import datetime, timedelta
import os
from playsound import playsound
import random

# Configure the serial port and baud rate
ser = serial.Serial('/dev/cu.usbserial-110', 115200, timeout=1)

# Define a function to determine the current week number
def get_week_number(date):
    start_date = datetime(2024, 7, 24)  # Change to the start date of your term YYYY-MM-DD
    return ((date - start_date).days // 7) + 1

# Specify the path for the CSV file
output_path_csv = os.path.join(os.getcwd(), 'DESN2804_attendance.csv')
print(f"CSV file will be saved to: {output_path_csv}")

# Load or create the attendance DataFrame
if os.path.exists(output_path_csv):
    df = pd.read_csv(output_path_csv)
else:
    df = pd.DataFrame(columns=['QR Code', 'Class'])

# Variables for debounce mechanism
last_qr_code = None
last_scan_time = datetime.min
debounce_time = timedelta(seconds=10)  # Adjust debounce time as needed

# Define your list of sound files
sound_files = [
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/bonus.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/BrassFanfare.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/Cossack.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/electronicBeep_win.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/FirstPrize.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/Gameshow.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/Organ.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/RiseKing.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/SecondPrizeWinner.mp3',
'/Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/Sounds/YouWin.mp3'
]

try:
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line and line.startswith("Found '") and line.endswith("'"):
            qr_data = line[7:-1]  # Extract the QR code content
            current_time = datetime.now()
            time_str = current_time.strftime('%H:%M')

            # Parse the QR code data
            class_name, qr_code = qr_data.split(':')
            
            # Debugging output for debounce mechanism
            print(f"Current QR Data: {qr_data}")
            print(f"Last QR Code: {last_qr_code}")
            print(f"Current Time: {current_time}")
            print(f"Last Scan Time: {last_scan_time}")
            print(f"Time Since Last Scan: {current_time - last_scan_time}")

            # Check for debounce
            if qr_data != last_qr_code or (current_time - last_scan_time) > debounce_time:
                last_qr_code = qr_data
                last_scan_time = current_time
                print(f"Processing QR code: {qr_code} for class {class_name}")  # Debugging statement

                timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S')
                week_number = get_week_number(current_time)
                week_col = f'Week {week_number}'
                time_col = f'Week {week_number} Time'
                print(f"Received QR data: {qr_data}")  # Debugging statement

                # Check if the QR code is already in the DataFrame
                if not ((df['QR Code'] == qr_code) & (df['Class'] == class_name)).any():
                    new_entry = pd.DataFrame({'QR Code': [qr_code], 'Class': [class_name]})
                    df = pd.concat([df, new_entry], ignore_index=True)
                    print(f"Added new entry: {qr_code} for class {class_name}")  # Debugging statement
                
                # Add new columns for the week if they don't exist
                if week_col not in df.columns:
                    df[week_col] = 'Absent'  # Default to 'Absent' for all initially
                    print(f"Added new column for {week_col}")  # Debugging statement

                if time_col not in df.columns:
                    df[time_col] = ''  # Default to empty for time initially
                    print(f"Added new column for {time_col}")  # Debugging statement
                
                # Mark the student as present and log the time
                df.loc[(df['QR Code'] == qr_code) & (df['Class'] == class_name), week_col] = 'Present'
                df.loc[(df['QR Code'] == qr_code) & (df['Class'] == class_name), time_col] = time_str
                print(f"Marked {qr_code} as present for {week_col} at {time_str}")  # Debugging statement
                
                print("Saving to CSV...")  # Debugging statement before saving
                df.to_csv(output_path_csv, index=False)
                print("CSV file saved.")  # Debugging statement after saving
                print(f'Logged: {qr_code} at {timestamp} for {week_col} with time {time_str}')
                
                  # Play a random sound
                sound_file = random.choice(sound_files)
                playsound(sound_file)
            else:
                print(f"Debounce active: Ignoring QR code {qr_code}")
except KeyboardInterrupt:
    print("\nInterrupt received. Saving CSV file and exiting...")
    df.to_csv(output_path_csv, index=False)
    print("CSV file saved.")

CSV file will be saved to: /Users/jbs939/Documents/StudioRepo/Attenderly/notebooks/DESN2804_attendance.csv
Current QR Data: 10AM:Sophie L
Last QR Code: None
Current Time: 2024-07-24 14:14:43.762513
Last Scan Time: 0001-01-01 00:00:00
Time Since Last Scan: 739090 days, 14:14:43.762513
Processing QR code: Sophie L for class 10AM
Received QR data: 10AM:Sophie L
Added new entry: Sophie L for class 10AM
Added new column for Week 1
Added new column for Week 1 Time
Marked Sophie L as present for Week 1 at 14:14
Saving to CSV...
CSV file saved.
Logged: Sophie L at 2024-07-24 14:14:43 for Week 1 with time 14:14
Current QR Data: 10AM:Sophie L
Last QR Code: 10AM:Sophie L
Current Time: 2024-07-24 14:14:47.473213
Last Scan Time: 2024-07-24 14:14:43.762513
Time Since Last Scan: 0:00:03.710700
Debounce active: Ignoring QR code Sophie L
Current QR Data: 10AM:Sophie L
Last QR Code: 10AM:Sophie L
Current Time: 2024-07-24 14:14:47.474583
Last Scan Time: 2024-07-24 14:14:43.762513
Time Since Last Scan: 0: